In [2]:
import csv
import pandas as pd
import numpy as np
import awswrangler as wr
from datetime import datetime, timedelta
import json
import gzip
from io import BytesIO, TextIOWrapper
import boto3
import dask.dataframe as dd
import plotly.express as px
import seaborn as sns

In [19]:
pos_rooftop_day=wr.athena.read_sql_query("select * from pos_rooftop_day", database="dunkin_db", keep_files=False)

In [21]:
pos_rooftop_day['gross_sales_amt'] = pos_rooftop_day[['gross_sales_amt']].fillna(0.00)
pos_rooftop_day['net_sales_amt'] = pos_rooftop_day[['net_sales_amt']].fillna(0.00)
pos_rooftop_day['check_cnt'] = pos_rooftop_day[['check_cnt']].fillna(0.00)

In [10]:
pos_roof_raw_missing=pos_rooftop_day[pos_rooftop_day.isna()].copy()

In [22]:
pos_rooftop_day.head()

,calendar_date,pfcntr,dma_name,dma_code,week_end_sat,ordr_type_name,gross_sales_amt,net_sales_amt,check_cnt
0,2019-06-02,310434,SPRINGFIELD MA,543,2019-06-08,drive_thru,2380.089844,2325.649902,476.0
1,2019-06-06,310434,SPRINGFIELD MA,543,2019-06-08,otg_walk_in,19.240000,19.240000,6.0
2,2019-06-06,310434,SPRINGFIELD MA,543,2019-06-08,otg_dt,11.560000,11.560000,4.0
3,2019-06-06,310434,SPRINGFIELD MA,543,2019-06-08,eat_in,1121.320068,1079.680054,239.0
4,2019-06-06,310434,SPRINGFIELD MA,543,2019-06-08,drive_thru,3002.469971,2932.780029,698.0


In [23]:
pos_rooftop_day[['calendar_date']].isna().sum()

calendar_date    0
dtype: int64

In [24]:
pos_rooftop_day[['pfcntr']].isna().sum()

pfcntr    0
dtype: int64

In [25]:
pos_rooftop_day['ordr_type_name'].isna().sum()

2978727

In [26]:
pos_rooftop_day[pos_rooftop_day['ordr_type_name'].isna()]

,calendar_date,pfcntr,dma_name,dma_code,week_end_sat,ordr_type_name,gross_sales_amt,net_sales_amt,check_cnt
9,2019-10-01,310434,SPRINGFIELD MA,543,2019-10-05,<NA>,0.0,0.0,0.0
10,2019-11-01,310434,SPRINGFIELD MA,543,2019-11-02,<NA>,0.0,0.0,0.0
11,2019-11-05,310434,SPRINGFIELD MA,543,2019-11-09,<NA>,0.0,0.0,0.0
133,2019-12-07,310435,PROVIDENCE RI,521,2019-12-07,<NA>,0.0,0.0,0.0
134,2019-12-11,310435,PROVIDENCE RI,521,2019-12-14,<NA>,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
39923702,2019-12-24,353236,METRO NY,501,2019-12-28,<NA>,0.0,0.0,0.0
39923703,2019-12-24,353236,METRO NY,501,2019-12-28,<NA>,0.0,0.0,0.0
39923704,2019-12-28,353236,METRO NY,501,2019-12-28,<NA>,0.0,0.0,0.0
39923705,2019-12-28,353236,METRO NY,501,2019-12-28,<NA>,0.0,0.0,0.0


In [27]:
pos_roof_raw_missing[['calendar_date']].isna().sum()

NameError: name 'pos_roof_raw_missing' is not defined

In [8]:
pos_roof_raw_missing[['pfcntr']].isna().sum()

pfcntr    131781478
dtype: int64

In [17]:
pos_rooftop_day.to_csv("pos_rooftop_day.csv.gz", index=False, header=True, 
                       encoding="utf-8", compression="gzip", sep="\t")

In [23]:
pos_rooftop_day=pos_rooftop_day.compute()

In [24]:
pos_rooftop_day.columns

Index(['calendar_date', 'pfcntr', 'dma_name', 'dma_code', 'week_end_sat',
       'ordr_type_name', 'gross_sales_amt', 'net_sales_amt', 'check_cnt'],
      dtype='object')

In [28]:
order_channel_week=pos_rooftop_day.groupby(['ordr_type_name','week_end_sat']).agg(
                                        gross_sales_amt = pd.NamedAgg("gross_sales_amt", "sum"),
                                        net_sales_amt = pd.NamedAgg("net_sales_amt", "sum"),
                                        check_cnt = pd.NamedAgg("check_cnt", "sum"))

In [66]:
pos_rooftop_week=pd.DataFrame(pos_rooftop_week.to_records())

In [67]:
pos_rooftop_week.head()

,ordr_type_name,week_end_sat,gross_sales_amt,net_sales_amt,check_cnt
0,br_delivery,2019-08-03,9.890000,9.890000,1.0
1,br_delivery,2020-08-01,0.000000,0.000000,0.0
2,college_card,2018-01-06,273.550018,272.570007,60.0
3,college_card,2018-01-13,3620.250000,3595.709961,967.0
4,college_card,2018-01-20,6875.229980,6846.140137,1765.0


In [68]:
pos_rooftop_week.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2439 entries, 0 to 2438
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ordr_type_name   2439 non-null   object 
 1   week_end_sat     2439 non-null   object 
 2   gross_sales_amt  2439 non-null   float32
 3   net_sales_amt    2439 non-null   float32
 4   check_cnt        2439 non-null   float32
dtypes: float32(3), object(2)
memory usage: 66.8+ KB


In [80]:
pos_rooftop_week.to_csv("pos_rooftop_week.csv", index=False, header=True, encoding="utf-8", sep="\t" )

In [37]:
pos_rooftop_week['ordr_type_name'].nunique()

23

In [39]:
pos_rooftop_week['week_end_sat']=pd.to_datetime(pos_rooftop_week['week_end_sat'])

In [41]:
from matplotlib import pyplot as plt

In [47]:
import dash
import dash_core_components as dcc
import dash_html_components as html

In [53]:
pd.options.plotting.backend="plotly"

In [78]:
fig=px.line(pos_rooftop_week, x="week_end_sat", y="check_cnt", color="ordr_type_name", height=800, width=1500, 
            line_group="ordr_type_name")
fig=fig.update_traces(mode='markers+lines')

In [79]:
app = dash.Dash(__name__)

app.layout = html.Div([
    dcc.Graph(id="graph", figure=fig),
])

if __name__ == '__main__':
    app.run_server(debug=True, use_reloader=False)

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


In [63]:
pos_rooftop_month=pos_rooftop_week.groupby(['pfcntr','ordr_type_name','month']).agg(
                                        gross_sales_amt = pd.NamedAgg("gross_sales_amt", "sum"),
                                        net_sales_amt = pd.NamedAgg("net_sales_amt", "sum"),
                                        check_cnt = pd.NamedAgg("check_cnt", "sum"))
pos_rooftop_month=pd.DataFrame(pos_rooftop_month.to_records())

In [64]:
pos_rooftop_month.to_csv("pos_rooftop_month.csv", index=False, header=True, encoding="utf-8", sep="\t")

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on
